In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
df_csv=pd.read_csv('/kaggle/input/online-retail-ii-uci/online_retail_II.csv')
df_csv['InvoiceDate']= pd.to_datetime(df_csv['InvoiceDate'])
df_csv['date']=df_csv['InvoiceDate'].dt.date
df_csv['date']= pd.to_datetime(df_csv['date'])
df_csv['month']=df_csv['InvoiceDate'].dt.month
df_csv['Sales']=df_csv['Quantity']*df_csv['Price']

df_csv.info() #  data shape, data types, counts and missing values
# ----------------------Answer-----------------
# Description and Customer ID fields have missing data
# 6885 Customer ID 'NA' and sales zero cases excluded from analysis 
# outliers of sales, price and quantity to be excluded from analysis



# ----------------------Answer-----------------

#1. Conduct exploratory data analysis and present your insights in a power point deck.
#2. Forecast the sales of the store for the 4 weeks in the month of December 2011.

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_csv.isna().any() #missing values in string columns only, no issue with numerical columns

In [ ]:
df_csv.nunique()
#df_csv['date'].unique()
#number of descrption values are more than stock codes, i.e. more than one name for a stock 

In [ ]:
df_csv.describe().T # describe basic stats
sdplusmean= df_csv.describe().T
sdplusmean['3SD']=sdplusmean['mean']+3*sdplusmean['std']
sdplusmean
#99.7% of the data will be within the 3SD (figures round off)
#Sales3SD= +-896
#Quantity3SD= +-528
#Price3SD= +-376

In [ ]:
equalzero=df_csv[df_csv['Sales']==0]
plt.hist(equalzero['Sales'],
         bins = int(50))
plt.xlim(xmin=-1, xmax = 1)

In [ ]:
equalzero.count()

In [ ]:
Notzero=df_csv[df_csv['Sales']!=0]
plt.hist(Notzero['Sales'], color = 'blue', edgecolor = 'black',
         bins = int(2500))
plt.xlim(xmin=-500, xmax = 500)
#*****----- all not zero sales are within -200 and +400 range----******

In [ ]:
Notzero=df_csv[df_csv['Sales']!=0]

plt.hist(Notzero['Sales'], color = 'blue', edgecolor = 'black',
         bins = int(10000))
plt.xlim(xmin=-200, xmax = 400)
#*****----- all not zero sales are within -200 and +400 range----******

In [ ]:
Notzero.count()

In [ ]:
greaterthanzero= df_csv[df_csv['Sales']>0]
greaterthanzero.count()

In [ ]:
lesthanzero=df_csv[df_csv['Sales']<0]
lesthanzero.count()

In [ ]:
pd.date_range(start = '2009-12-01', end = '2011-12-09' )#.difference(df_csv['date'])
#01/12/2009 to 09/12/2011 is 739 days but the dates are 604 dates only as per last block
# ******-----so dates are not continuos, maybe holidays, etc -----*****

In [ ]:
ss=pd.date_range(start = '2009-12-01', end = '2011-12-09' ).difference(df_csv['date']) #missing dates 135
ss

In [ ]:
StockCheck =df_csv[['StockCode','Sales','Description']].groupby(['StockCode','Description']).agg({'Sales': [sum]})
StockCheck
uniqdesc=df_csv['Description'].unique()
np_arr1 = np.array(uniqdesc)
data_arr1 = np.reshape(np_arr1, (41, 139))
#pd.DataFrame(data_arr1)


In [ ]:
np_arr = np.array(ss)
data_arr = np.reshape(np_arr, (15, 9))
ss1 = pd.DataFrame(data_arr, columns=["a", "b", "c", "d","e","f","g","h","i"])
ss1
#ss-ss[0]
# ******-----24thDec to 3rdJan we have no orders in last two years, can be holiday time-----*****

In [ ]:
df_csv[(df_csv['Customer ID'].isna())]

#*******-------customer ID is na in 243,007 cases------******, df_csv[(df_csv['Customer ID'].isna())].count() 

In [ ]:
df_csv[(df_csv['Customer ID'].isna())& (df_csv['Sales']<=0)] 
#*****-----6885 cases Zeros sale cases (include 4382 Description NA cases)-----*****

In [ ]:
Custid_na= df_csv[(df_csv['Customer ID'].isna())& (df_csv['Sales']>0) ]
Custid_na
#Custid_na['Sales'].sum()

#********--------236122 rows have sales greater than zero but the customer ID is blank--------*******

In [ ]:
df_csv[(df_csv['Description'].isna())] # decription is na in 4382 and in all these cases, customer id is also na
#sum of sales is zero so let's exclude from sales analysis

In [ ]:
description_na=df_csv[(df_csv['Description'].isna())]
description_na['Sales'].sum() 
#*******------sum of sales is zero for 4382'decription na'cases,customer id is also na, so let's exclude from analysis of sales

In [ ]:
df_csv[(df_csv.Sales>=896)|(df_csv.Sales<=-896)|(df_csv.Sales==0)]

In [ ]:
df_csv[df_csv['Sales']==0].count()

In [ ]:
Sales_data=df_csv[(df_csv.Sales<=896)&(df_csv.Sales>=-896)&(df_csv.Sales!=0)]#outliers removed Sales3SD= +-896
Sales_data

In [ ]:
Sales_data.info() # data looks clean now only Customer ID is null in few cases where sales are non-zero

In [ ]:
Sales_data.count() # data looks clean now only Customer ID is null in few cases where sales are non-zero

In [ ]:
Sales_data['Sales'].plot.kde()

In [ ]:
plt.hist(Sales_data['Sales'], bins=int(500))
plt.xlim(xmin=-400,xmax=400)

In [ ]:
plt.hist(Sales_data['Sales'], bins=int(500))
plt.xlim(xmin=-50,xmax=100)

In [ ]:
Sales_data[Sales_data['Sales']<=0] # sales returns 19307

In [ ]:
Sales_data[Sales_data.Invoice.str.contains('C')] # sales returns as per Invoice 'C'... 19308

In [ ]:
Sales_data[(Sales_data.Invoice.str.contains('C')) & (Sales_data['Sales']>=0)]
# ******---one row incorrectly marked as C in invoice or Quantity not marked as -ve issue, exclude from analysis----******

In [ ]:
Sales_data_clean=Sales_data.drop([76799])
Sales_data_clean

In [ ]:
Sales_data_clean['MonthYear']= Sales_data_clean['InvoiceDate'].apply(lambda x: x.strftime('%B-%Y')) 
Sales_data_clean['WeekYear']= Sales_data_clean['InvoiceDate'].apply(lambda x: x.strftime('%V-%Y')) 
Sales_data_clean['Year']= Sales_data_clean['InvoiceDate'].apply(lambda x: x.strftime('%Y')) 
Sales_data_clean

In [ ]:
Sales_data_clean.info()

In [ ]:
Sales_data_clean.describe().T

In [ ]:
#daily sales
from matplotlib.pyplot import figure

figure(figsize=(35, 6), dpi=80)
sales_daily = Sales_data_clean[Sales_data_clean['Sales']>0]
#sales_daily = Sales_data_clean
sales_daily.groupby('WeekYear').sum()

y= sales_daily['Sales']
plt.scatter(sales_daily['WeekYear'],y)
plt.xticks(rotation=45)


#daily returns
from matplotlib.pyplot import figure

figure(figsize=(35, 6), dpi=80)
sales_daily_ret = Sales_data_clean[Sales_data_clean['Sales']<=0]
sales_daily_ret.groupby('WeekYear').sum()

y= sales_daily_ret['Sales']
plt.scatter(sales_daily_ret['WeekYear'],y)
plt.xticks(rotation=45)

#total Year
from matplotlib.pyplot import figure

figure(figsize=(35, 6), dpi=80)
sales_tot = Sales_data_clean
sales_tot.groupby('WeekYear').sum()

y= sales_tot['Sales']
plt.scatter(sales_tot['WeekYear'],y)
plt.xticks(rotation=45)

#total week year
sales_tot_week = Sales_data_clean[(Sales_data_clean['month']>11)]
sales_tot_week.groupby('WeekYear').sum()

y= sales_tot_week['Sales']
plt.scatter(sales_tot_week['WeekYear'],y)
plt.xticks(rotation=45)


In [ ]:
#plt.figure()
#sns.pairplot(Sales_data_clean)
#display(plt.show())


In [ ]:
Sales_data_clean.info()

In [ ]:
Sales_data_clean.nunique()

In [ ]:
plt.figure()
from matplotlib.pyplot import figure
figure(figsize=(6, 4), dpi=30)
sales_pivot= Sales_data_clean.pivot_table(index='Country',values='Sales', aggfunc=np.sum)
sales_pivot.plot(kind='bar')

In [ ]:
plt.figure()
Sales_data_clean_2009=Sales_data_clean[(Sales_data_clean['Year']=='2009')]
sales_pivot_1= Sales_data_clean_2009.pivot_table(index='WeekYear',values='Sales', aggfunc=np.sum)
sales_pivot_1


In [ ]:
plt.figure()
Sales_data_clean_2009=Sales_data_clean[(Sales_data_clean['Year']=='2009')]
sales_pivot_1= Sales_data_clean_2009.pivot_table(index='MonthYear',values='Sales', aggfunc=np.sum)


Sales_data_clean_2010=Sales_data_clean[(Sales_data_clean['Year']=='2010')]
sales_pivot_2= Sales_data_clean_2010.pivot_table(index='MonthYear',values='Sales', aggfunc=np.sum)

Sales_data_clean_2011=Sales_data_clean[(Sales_data_clean['Year']=='2011')]
sales_pivot_3= Sales_data_clean_2011.pivot_table(index='MonthYear',values='Sales', aggfunc=np.sum)

fig, ax = plt.subplots()
#ax1 = ax.twiny()
ax2 = ax.twiny()


#sales_pivot_1.plot(kind='bar',ax=ax1)
sales_pivot_2.plot(kind='bar',ax=ax)
sales_pivot_3.plot(kind='bar',ax=ax2,alpha=0.3, legend=False)

In [ ]:
#weekly moving average for original data
ts= Sales_data_clean.pivot_table(index='date',values='Sales', aggfunc=np.sum)
ts
moving_avg_1 = ts.rolling(7).mean()
moving_std_1 = ts.rolling(7).std()
plt.plot(ts)
plt.plot(moving_avg_1, color='red')
plt.plot(moving_std_1, color='green')
plt.xticks(rotation=45)

In [ ]:
#decompose data for trends and seasonality
from pandas import Series
from matplotlib import pyplot
from statsmodels.tsa.seasonal import seasonal_decompose
series = ts
result = seasonal_decompose(series, model='additive',period=7)
result.plot()
pyplot.show()

# indicates strong trends and seasonality

In [ ]:
#convert to log
ts_log = np.log(ts)
moving_avg_2 = ts_log.rolling(7).mean()
moving_std_2 = ts_log.rolling(7).std()
plt.plot(ts_log)
plt.plot(moving_avg_2, color='red')
plt.plot(moving_std_2, color='green')
plt.xticks(rotation=45)

In [ ]:
#decomposition
ts_log_moving_avg_diff = ts_log - moving_avg_2
ts_log_moving_avg_diff.head(10)

ts_log_diff = ts_log - ts_log.shift()
moving_avg_3 = ts_log_diff.rolling(7).mean()
moving_std_3 = ts_log_diff.rolling(7).std()
plt.plot(ts_log_diff)
plt.plot(moving_avg_3, color='red')
plt.plot(moving_std_3, color='orange')
plt.xticks(rotation=45)

In [ ]:
#remove nas from data to decompose
ts_log_diff
ts_log_diff_remna=ts_log_diff.dropna()
ts_log_diff_remna

#decompose the data to view the status post differencing
from pandas import Series
from matplotlib import pyplot
from statsmodels.tsa.seasonal import seasonal_decompose
series = ts_log_diff_remna
result = seasonal_decompose(series, model='additive',period=7)
result.plot()
pyplot.show()

In [ ]:
# auto correlation and partial autocorrelation original data

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# Import Data
df = ts

# Draw Plot
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(16,6), dpi= 80)
plot_acf(df.Sales.tolist(), ax=ax1, lags=50)
plot_pacf(df.Sales.tolist(), ax=ax2, lags=20)

# Decorate
# lighten the borders
ax1.spines["top"].set_alpha(.3); ax2.spines["top"].set_alpha(.3)
ax1.spines["bottom"].set_alpha(.3); ax2.spines["bottom"].set_alpha(.3)
ax1.spines["right"].set_alpha(.3); ax2.spines["right"].set_alpha(.3)
ax1.spines["left"].set_alpha(.3); ax2.spines["left"].set_alpha(.3)

# font size of tick labels
ax1.tick_params(axis='both', labelsize=12)
ax2.tick_params(axis='both', labelsize=12)
plt.show()

# adf for original data
from statsmodels.tsa.stattools import adfuller,acf, pacf
from statsmodels.tsa.arima_model import ARIMA

dftest= adfuller(ts,autolag='AIC')
dfoutput=pd.Series(dftest[0:4],index=['Test Statisitc','p-value','#Lags Used','Number of Observations Used'])

for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key]= value
print(dfoutput)  

# t-statistic -2.3>-2.5(90% confidence level) and p-value 0.16>0.05  
# null hypothesis cannot be rejected, so series is non-stationary

In [ ]:
# auto correlation and partial autocorrelation after differencing data

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# Import Data
df1 = ts_log_diff_remna

# Draw Plot
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(16,6), dpi= 80)
plot_acf(df1.Sales.tolist(), ax=ax1, lags=50)
plot_pacf(df1.Sales.tolist(), ax=ax2, lags=20)

# Decorate
# lighten the borders
ax1.spines["top"].set_alpha(.3); ax2.spines["top"].set_alpha(.3)
ax1.spines["bottom"].set_alpha(.3); ax2.spines["bottom"].set_alpha(.3)
ax1.spines["right"].set_alpha(.3); ax2.spines["right"].set_alpha(.3)
ax1.spines["left"].set_alpha(.3); ax2.spines["left"].set_alpha(.3)

# font size of tick labels
ax1.tick_params(axis='both', labelsize=12)
ax2.tick_params(axis='both', labelsize=12)
plt.show()

# adf after differencing data

from statsmodels.tsa.stattools import adfuller,acf, pacf
from statsmodels.tsa.arima_model import ARIMA

dftest= adfuller(ts_log_diff_remna,autolag='AIC')
dfoutput=pd.Series(dftest[0:4],index=['Test Statisitc','p-value','#Lags Used','Number of Observations Used'])

for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key]= value
print(dfoutput)

# p-value of '8.23*10^-15'<0.05 and t-value -8.96<-3.4(99% Confidence level) is also lower than critical values
# so null hypothesis can be rejected, log series after differencing is stationary

#This suggests that at least one level of differencing is required. 
#The d parameter in our ARIMA model should at least be a value of 1.

In [ ]:
#lag values for the Autoregression (AR) and Moving Average (MA) parameters, p and q.
#looking the acf and pacf models let's start with pand q values of 0 and 1 , we'll use confirguration ARIMA(0,1,2)

from statsmodels.tsa.arima_model import ARIMA
model= ARIMA(ts_log, order=(7,1,0))
results_ARIMA= model.fit(disp=-1)

from matplotlib.pyplot import figure
figure(figsize=(35, 6), dpi=90)

plt.plot(ts_log_moving_avg_diff)
plt.plot(results_ARIMA.fittedvalues,color='red')
plt.xticks(rotation=45)
results_ARIMA.summary()

In [ ]:
predictions_ARIMA_diff=pd.Series(results_ARIMA.fittedvalues,copy=True)
predictions_ARIMA_diff

In [ ]:
predictions_ARIMA_diff_cumsum= predictions_ARIMA_diff.cumsum()
predictions_ARIMA_diff_cumsum.head(10)

In [ ]:
predictions_ARIMA_log = pd.Series(ts_log.iloc[0],index=ts_log.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
predictions_ARIMA_log

In [ ]:
predictions_ARIMA=np.exp(predictions_ARIMA_log)
plt.plot(ts_log)
#plt.plot(predictions_ARIMA)
#plt.title('RMSE: %.4f'%np.sqrt(sum(predictions_ARIMA-ts)**2)/len(ts))
#np.sqrt(sum(predictions_ARIMA-ts)**2)/len(ts)
plt.xticks(rotation=45)

In [ ]:
import statsmodels.api as sm
model=sm.tsa.statespace.SARIMAX(ts_log['Sales'],order=(7, 1, 0),seasonal_order=(2,1,1,12))
results=model.fit()
ts_log['forecast']=results.predict(start=90,end=590,dynamic=False)
ts_log[['Sales','forecast']].plot(figsize=(12,8))

results.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
from pandas.tseries.offsets import DateOffset
future_dates=[ts_log.index[-1]+ DateOffset(n=x)for x in range(0,28)]

ts_log_1= results.predict(start = 591, end = 617, dynamic= False)
ts_log_1.index=future_dates[1:]
ts_log_1.columns=ts_log.columns

ts_log_2=pd.concat([ts_log,ts_log_1])
ts_log_2.columns=['Sales','Predict','Forecast']
ts_log_2[['Sales','Forecast']].plot(figsize=(12,8))


In [ ]:
import matplotlib.pyplot as plt
predictions_SARIMAX=np.exp(results.predict(start = 591, end = 617, dynamic= False))
predictions_SARIMAX.index=future_dates[1:]
predictions_SARIMAX.columns=['Date','Forecast']
predictions_SARIMAX

plt.plot(ts)
plt.plot(predictions_SARIMAX)
plt.xticks(rotation=45)

In [ ]:
ts.tail(5).plot()
predictions_SARIMAX.head(14).plot()
December_2011=pd.concat([ts.tail(5),predictions_SARIMAX.head(14)])

In [ ]:
December_2011_remna=December_2011.fillna(0)
December_2011_remna
December_2011_remna['TotalForecast']=December_2011_remna['Sales']+December_2011_remna[0]
December_2011_remna
December_2011_remna['date']= December_2011_remna.index#.apply(lambda x: x.strftime('%V-%Y')) 
December_2011_remna
December_2011_remna['WeekYear']=December_2011_remna['date'].apply(lambda x: x.strftime('%V-%Y'))
December_2011_remna

In [ ]:
December_2011_remna_pivot= December_2011_remna.pivot_table(index='WeekYear',values='TotalForecast', aggfunc=np.sum)
December_2011_remna_pivot,December_2011_remna_pivot.plot(kind='bar')
print('Only three weeks predicted.')
print('Week 52 sales to be zero as it falls b/w 24th Dec and 3rd Jan, when there are usullay no orders')

In [ ]:
df_csv['Week']= df_csv['InvoiceDate'].apply(lambda x: x.strftime('%V')) 
df_csv['Year']= df_csv['InvoiceDate'].apply(lambda x: x.strftime('%Y'))
df_csv['WeekYear']= df_csv['InvoiceDate'].apply(lambda x: x.strftime('%V-%Y')) 


In [ ]:
# comparison with last three years
df_csv_Piv_2009=df_csv[(df_csv['month']>11)&(df_csv['Year']=='2009')]
df_csv_Piv_2010=df_csv[(df_csv['month']>11)&(df_csv['Year']=='2010')]

df_csv_Piv_2009_pivot= df_csv_Piv_2009.pivot_table(index='WeekYear',values='Sales', aggfunc=np.sum)
df_csv_Piv_2010_pivot=df_csv_Piv_2010.pivot_table(index='WeekYear',values='Sales', aggfunc=np.sum)

df_csv_Piv_2009_pivot ,df_csv_Piv_2010_pivot, December_2011_remna_pivot

In [ ]:
December_2011_remna_pivot= December_2011_remna.pivot_table(index='WeekYear',values='TotalForecast', aggfunc=np.sum)
December_2011_remna_pivot
df_csv_Piv_2009_pivot
df_csv_Piv_2010_pivot
df_csv_Piv_2009_pivot.plot(kind='bar') 
df_csv_Piv_2010_pivot.plot(kind='bar')
December_2011_remna_pivot.plot(kind='bar')


In [ ]:
#results.resid.plot()

#ret_to_Sales_vol=sales_daily_ret['Sales'].count()/Sales_data_clean['Sales'].count()
#ret_to_Sales_vol*100
#*******--------- 1.82 % of total orders are returns in terms of volume------*********

#ret_to_Sales_val=-sales_daily_ret['Sales'].sum()/Sales_data_clean['Sales'].sum()
#ret_to_Sales_val*100
#*******--------- 2.92 % of total orders are returns in terms of value------*********



#1 Totalsales= df_csv.Sales.sum()
#Totalsales

#2 returns=df_csv[df_csv.Sales<=0]
#returns['Sales'].sum()

#3 Totalsales+returns['Sales'].sum() # 17 million net sales after returns

#4 ret_sale= -returns['Sales'].sum()/Totalsales
#ret_sale # *****----- returns are 8% in terms of value of the sales ...... analysis without any exclusions -----*****

#5 lessthanzero['Sales'].count()/df_csv['Sales'].count()
# *****----- returns are 1.8% in terms of counts of the sales.... -----*****

In [ ]:
#df_csv[df_csv['date'] == '29-01-2010'& df_csv['Price'].max()]
#df_csv[(df_csv.Price <=0)].groupby('date').agg({'Quantity':['sum']})
#Sales_data[(Sales_data.Price <0)] # negative price denotes bad debt/write offs, so to be excluded

#import plotly_express as px
#date_price = px.area(df_csv, x='InvoiceDate', y='Price')
#date_price.show()